### Integrantes: <a class="tocSkip"></a>

**Ignacio Castro**

**Enzo Meneses**


### Instrucciones generales <a class="tocSkip"></a>

1. Forme un grupo de **máximo dos estudiantes**
1. Versione su trabajo usando un **repositorio <font color="red">privado</font> de github**. Agregue a su compañero y a su profesor (usuario github: phuijse) en la pestaña *Settings/Manage access*. No se aceptarán consultas si la tarea no está en github. No se evaluarán tareas que no estén en github.
1. Se evaluará el **resultado, la profundidad de su análisis y la calidad/orden de sus códigos** en base al último commit antes de la fecha y hora de entrega". Se bonificará a quienes muestren un método de trabajo incremental y ordenado según el histórico de *commits*
1. Sean honestos, ríganse por el [código de ética de la ACM](https://www.acm.org/about-acm/code-of-ethics-in-spanish)



# Tarea 2: Filtrado de señales

## Introducción


Un **electrocardiograma (ECG)** es un gráfico de voltaje (amplitud) en función del tiempo correspondiente a la actividad eléctrica del corazón. Cada etapa del ciclo cardíaco es representado por una letra: **P, Q, R, S y T**. La siguiente figura muestra un esquema de un ECG con dos pulsos.

<img src="cardio.png" width="600">


El ECG se usa para hacer diagnósticos y en particular para analizar la existencia de anormalidades en el ciclo cardiaco. La **frecuencia cardiaca**, es decir el inverso del intervalo RR, de un humano normal en reposo está en el rango de **60 y 100 latidos por minuto**



## Enunciado

El hospital base de Valdivia tiene un electrocardiógrafo que mide la ECG en miliVolts [mV] a una frecuencia de muestreo de $360$ [Hz]

Durante el proceso de captura ciertos errores provocaron que la señal se contamine con 

- **(A)** Un ruido asociado al sistema eléctrico
- **(B)** Un ruido de baja frecuencia causado por un desperfecto del sensor
- **(C)** Un ruido blanco de origen térmico 

Esto complica el análisis del médico por lo que el hospital base los contrata a ustedes para **diseñar un filtro que limpie la señal ECG**

Resuelva las actividades que se piden a continuación, respondiendo y analizando donde corresponda

In [1]:
import numpy as np
import pickle
%matplotlib notebook
import matplotlib.pylab as plt
import scipy.signal
from scipy import fftpack

with open("ECG.pkl", "rb") as f:
    data = pickle.load(f)

Fs = 360
time = np.arange(len(data))/Fs
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.plot(time, data)
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Amplitud [mV]');

<IPython.core.display.Javascript object>

## Análisis (15%)

- Observe los primeros dos segundos de la señal, compare con el esquema mostrado en la introducción y discuta sobre los efectos causados por los ruidos **A**, **B** y **C** mencionados en el enunciado. Además responda  ¿Qué es un ruido blanco y que tipo de efecto introduce en la señal y en su espectro? ¿Qué es una tendencia y que tipo de efecto introduce en la señal y su espectro? 
- Calcule y muestre gráficamente el valor absoluto de la transformada de Fourier de la señal en función de la frecuencia y su espectrograma en función del tiempo y la frecuencia. Para el espectrograma decida el largo de ventana, traslape y tipo de ventana. Justifique su elección tomando en consideración el rango de frecuencias de un paciente normal y los *trade-off* aprendidos en clases
- En base a la señal, su espectro de magnitud y su espectrograma identifique la frecuencia central o el rango de frecuencias asociada a las tres fuentes de ruido. ¿En qué rango de frecuencias (Hz) se espera encontrar la señal de ritmo cardíaco de un humano normal en reposo?


In [2]:
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.plot(time[:2*Fs], data[:2*Fs])
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Amplitud [mV]');

<IPython.core.display.Javascript object>

En este caso podemos observar 3 tipos de ruido, el ruido A corresponde a un ruido asociado al sistema electrico y tendera a tener frecuencias cercanas a los 60Hz y dara a la señal una tendencia de aumento. El ruido B corresponde a un ruido de baja frecuencia cue creara picos e menor frecuencia pero mayor plitud en la señal. Finalmente el Corresponde a ruido blanco, que tiene una distribucion aleatoria y se distribuye por toda la señal.

El ruido blanco corresponde a variaciones irregulares o aleatorios en la señal observada, y en el espectro se ve como varios espacios de baja densidad espectral sin un orden especifico.

La tandencia se ve en la señal como un patron en su forma, y en el espectro se ve como una columna o fila de densidad espectral similar a la de la señal original.

In [3]:
def fourierfun(signal,Fs):
    data = fftpack.fft(signal)
    data_freq = fftpack.fftfreq(n=len(data), d=1/Fs)
    data_amp = np.abs(data)
    data_angle = np.angle(data)
    return data_freq,data_amp,data_angle

In [4]:
freqs,SA,SP=fourierfun(data,Fs)

fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.plot(fftpack.fftshift(freqs),fftpack.fftshift(SA))

<IPython.core.display.Javascript object>

In [5]:
# espectogrma
Nw = 600
beta = 8.6
freqs, times, Sxx = scipy.signal.spectrogram(data, fs=Fs, nperseg=Nw, 
                                                 noverlap=Nw//1.5, window=('kaiser', beta))
fig, ax = plt.subplots(figsize=(7, 5), tight_layout=True, sharex=True, sharey=True)
ax.pcolormesh(times, freqs, Sxx, cmap=plt.cm.Reds, shading='auto');
ax.set_ylabel('Frequencia [Hz]');
ax.set_xlabel('Tiempo [s]');

<IPython.core.display.Javascript object>

Elegimos la ventana Kaiser pues es una ventana muy adaptable debido a su valor Beta, lo que permite ajustar el grado de importancia que se le daran a los lobulos laterales o principal con solo modificar este valor para emular otros tipos de ventanas, sin requerir cambiar el resto de las especificaciones. El beta elegido fue el que encontramos empiricamente que ilustraba mejor el espectro, esta basado en el bet recomendado para aproximar una ventana de blackman segun la documentación de Scipy.

En cuanto al tamaño de ventana, este tambien lo encontramos empiricamente. Opinamos que 600 permite observar con suficiente detalle los distintos rangos de frecuencia, a la vez que no es tan grande que se pierda información.

**En base a la señal, su espectro de magnitud y su espectrograma identifique la frecuencia central o el rango de frecuencias asociada a las tres fuentes de ruido. ¿En qué rango de frecuencias (Hz) se espera encontrar la señal de ritmo cardíaco de un humano normal en reposo?**

Viendo el espectro de magnitud y el espectograma, podemos teorizar que el ruido B, de baja frecuencia y alta amplitud se encuentra cercano a la frecuencia 0 (haciendo zoom vemos valores entre 0.3 y 0.9 Hz). El ruido blanco C un poco más alejado de la frecuencia 0, con valores menores frecuencia menores a 30 Hz por lo que podemos observar en el espectograma. El ruido del sistema electrico A tiende a ubicarse cercano a los 60 Hz, entre 58-62 Hz.

Un humano sano en reposo tiene una frecuencia cardiaca entre 60 y 100 latidos por minuto.

## Filtrado FIR (50%)

Describa lo que es un filtro FIR, explique paso a paso el método de enventanado y describa los argumentos y el valor de retorno de la función `scipy.signal.firwin`. 

Luego use este método para diseñar y aplicar tres filtros sobre la señal de entrada. El primero debe filtrar el ruido **(A)**, el segundo el ruido **(B)** y el tercero el ruido **(C)**. Indique la(s) frecuencia(s) de corte y el tipo de filtro en cada caso. Justifique su elección en base a sus observaciones en el paso anterior.

Considere una ventana de Hamming y $1001$ coeficientes para el filtro

Por cada filtro:
- Muestre la respuesta al impulso y la respuesta en frecuencia resultantes
- Filtre la señal convolucionando con la respuesta al impulso
- Calcule la resta entre la señal original y la señal filtrada. ¿Qué espera observar en este caso? Luego muestre gráficamente el espectro de magnitud y el espectrograma de la señal restada. En base a lo que observa y a los gráficos del punto anterior discuta sobre la efectividad del filtro

Finalmente
- Filtre la señal convolucionando con los tres filtros. Muestre el resultado graficamente. 
- Muestre un acercamiento de los primeros tres segundos de la señal filtrada, reconozca la fase R y márquela con una linea roja punteada.
- Cree una función que mida el tiempo RR en la señal filtrada. Grafique la frecuencia cardiaca en función del tiempo. ¿Se encuentra la persona en el rango sano o no?

**¿Que es un filtro FIR?**

Los filtros FIR (Respuesta Finita al Impulso) el un filtro basado en un sistema reververante, es decir, al aplicar el filtro a una señal el output dependera no solo del valor actual, sino de los L valores anteriores, que seran multiplicados por distintos coeficientes segun el filtro de manera que tengan distinto peso sobre el ultimo resultado. Se llaman finitas porque solo puede operar elementos de N orden mientres 0<N<L, para los otros valores retornara 0, entonces tiene una duración finita el filtro.

**Explique el método de enventanado**

El enventanado consiste, como su nombre lo dice, en aplicar una función ventana a una señal para su procesamiento o análisis. La ventana es una función matematica que posee ciertas areas o dimensiones, y retornara 0 para elementos fuera de esta area o una función dependiendo de que tipo de ventana sea para elementos que estén dentro de ella. Luego la ventana se mueve por la señal, multiplicando los elementos que la sobrelapan por las dimensiones que tiene la ventana en dicha posición, y retornando el resultado.

**Describa los argumentos y el valor de retorno de la función "scipy.signal.firwin"**

    Parametros:
                numtaps: Largo del filtro (L+1).
                cutoff: Frecuencia de corte o arregle de frecuencias de corte. Debe estar entre 0 y fs/s.
                width: Ancho de la región de transición.
                window: Tipo de ventana que se desea usar.
                pass_zero: Especifica que tipo de filtro se quiere usar para el cutoff.
                scale: Escala de los coeficientes.
                nyq: Deprecado, usar fs.
                fs: Frecuencia de muestreo original.
    Retorna:
                h: Coeficientes de filtro FIR.
            




### Filtro A

In [6]:
L = 1001

# ruido asociado al sistema electrico 60 Hz aprox
#filtroA = scipy.signal.firwin(L, 62, window='hamming', pass_zero=True, fs=Fs)
#filtroA = scipy.signal.firwin(L, [58,62], window='hamming', pass_zero=True, fs=Fs)
filtroA = scipy.signal.firwin(L, 58, window='hamming', pass_zero=True, fs=Fs)

#Es un filtro pasabajos pues queremos filtrar el ruido de alta frecuencia del sistema eléctrico.

freq, H = scipy.signal.freqz(filtroA, fs=Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(filtroA)
ax[1].plot(freq, np.absolute(H))

<IPython.core.display.Javascript object>

In [7]:
# convolucion con filtro A
yA = scipy.signal.convolve(data, filtroA, mode='same')
YA = fftpack.rfft(yA)
freqs = fftpack.rfftfreq(n=len(yA), d=1/Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(time[:len(yA)], yA)
ax[1].plot(freqs, np.absolute(YA))

<IPython.core.display.Javascript object>

**Calcule la resta entre la señal original y la señal filtrada. ¿Qué espera observar en este caso?**

Al restar la senal filtrada de la original debería quedar lo que se filtro, en este caso ruido electrico de 60 Hz.

In [8]:
# resta con filtro A
rA = data[:len(yA)] - yA

freqs1, SA, SP=fourierfun(rA,Fs)
freqs2, times, Sxx = scipy.signal.spectrogram(rA, fs=Fs, nperseg=Nw, 
                                                 noverlap=Nw//1.5, window=('kaiser', beta))

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(fftpack.fftshift(freqs1),fftpack.fftshift(SA))
ax[1].pcolormesh(times, freqs2, Sxx, cmap=plt.cm.Reds, shading='auto');

<IPython.core.display.Javascript object>

**Discutir efectividad filtro A**

El filtro fue efectivo pues podemos ver que en la resta aparecen los elementos que deseabamos eliminar tanto en el espectograma como en el espectro de amplitud.


### Filtro B

In [9]:
# 1-1.6 latidos por segundo
# Ruido de baja frecuencia < 1 Hz
#filtroB = scipy.signal.firwin(L, [0.3, 0.9], window='hamming', pass_zero=False, fs=Fs)
filtroB = scipy.signal.firwin(L, 0.9, window='hamming', pass_zero=False, fs=Fs)

#Es un filtro pasa alto pues queremos filtrar el ruido de frecuencia baja.

freq, H = scipy.signal.freqz(filtroB, fs=Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(filtroB)
ax[1].plot(freq, np.absolute(H))

<IPython.core.display.Javascript object>

In [10]:
yB = scipy.signal.convolve(data, filtroB, mode='same')
YB = fftpack.rfft(yB)
freqs = fftpack.rfftfreq(n=len(yB), d=1/Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(time[:len(yB)], yB)
ax[1].plot(freqs, np.absolute(YB))

<IPython.core.display.Javascript object>

**Calcule la resta entre la señal original y la señal filtrada. ¿Qué espera observar en este caso?**

Al restar la senal filtrada de la original debería quedar lo que se filtro, en este caso el ruido de baja frecuencia, entonces en este caso deberian quedar las senales < 0.9 Hz.

In [11]:
# resta filtro B
rB = data[:len(yB)] - yB

freqs1, SA, SP=fourierfun(rB,Fs)
freqs2, times, Sxx = scipy.signal.spectrogram(rB, fs=Fs, nperseg=Nw, 
                                                 noverlap=Nw//1.5, window=('kaiser', beta))

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(fftpack.fftshift(freqs1),fftpack.fftshift(SA))
ax[1].pcolormesh(times, freqs2, Sxx, cmap=plt.cm.Reds, shading='auto');

<IPython.core.display.Javascript object>

**Discutir efectividad filtro B**

Podemos ver en ambos gráficos de la resta que aparece el ruido de menor frecuencia (aunque quiza sea necesario haccer zoom para verlo en el espectograma), por lo que podemos concluir que el filtro lo elimino exitosamente de la señal real.


### Filtro C

In [12]:
# ruido blanco ???

#filtroC = scipy.signal.firwin(L, [0.9, 58], window='hamming', pass_zero=False, fs=Fs)
filtroC = scipy.signal.firwin(L, [28,32], window='hamming', pass_zero='bandstop', fs=Fs)

#Es un filtro rechazabanda pues el ruido blanco se encuentra distribuido en un rango mas amplio.

freq, H = scipy.signal.freqz(filtroC, fs=Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(filtroC)
ax[1].plot(freq, np.absolute(H))



<IPython.core.display.Javascript object>

In [13]:
yC = scipy.signal.convolve(data, filtroC, mode='same')
YC = fftpack.rfft(yC)
freqs = fftpack.rfftfreq(n=len(yC), d=1/Fs)

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(time[:len(yC)], yC)
ax[1].plot(freqs, np.absolute(YC))

<IPython.core.display.Javascript object>

**Calcule la resta entre la señal original y la señal filtrada. ¿Qué espera observar en este caso?**

Al restar la senal filtrada de la original debería quedar lo que se filtro, en este caso una nube de ruido blanco.

In [14]:

rC = data[:len(yC)] - yC

freqs1, SA, SP=fourierfun(rC,Fs)
freqs2, times, Sxx = scipy.signal.spectrogram(rC, fs=Fs, nperseg=Nw, 
                                                 noverlap=Nw//1.5, window=('kaiser', beta))

fig, ax = plt.subplots(1,2, figsize=(7, 3), tight_layout=True)
ax[0].plot(fftpack.fftshift(freqs1),fftpack.fftshift(SA))
ax[1].pcolormesh(times, freqs2, Sxx, cmap=plt.cm.Reds, shading='auto');

<IPython.core.display.Javascript object>

**Discutir efectividad filtro C**

Podemos ver en ambos gráficos de la resta que pudimos efectivamente disminuir parte del rudio blanco usando el filtro rechaza bandas.

In [15]:
# filtrar
fil = scipy.signal.convolve(data, filtroA, mode='same')
fil = scipy.signal.convolve(fil, filtroB, mode='same')
#fil=fil_fir
fil = scipy.signal.convolve(fil, filtroC, mode='same')

fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.plot(time, fil)
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Amplitud [mV]');


<IPython.core.display.Javascript object>

In [16]:
# mostrar 3 seg, marcar fase R
fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
#ax.plot(time[3*Fs:12*Fs], fil[3*Fs:12*Fs])
ax.plot(time[:3*Fs], fil[:3*Fs])
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Amplitud [mV]');


<IPython.core.display.Javascript object>

In [17]:
# funcion que mida tiempo RR
# arreglar
def RR(signal, Fs):
    t = np.argwhere(signal > 1)
    return (t[3] - t[0]) / Fs
RR(fil, Fs)


# funcion que mida tiempo RR
# arreglar


array([0.81666667])

## Filtrado IIR (35%)

Describa los filtros IIR y compárelo con los filtros FIR. Describa los argumentos y el valor de retorno de la función `scipy.signal.iirfilter`.

Luego use este método para diseñar tres filtros. Los filtros deben cumplir el mismo propósito que en el caso anterior. Considere filtros IIR de tipo `butterworth` y todos del mismo orden. Aplique los filtros sobre la señal de entrada usando la función `scipy.signal.lfilter`.

El objetivo es encontrar el mínimo orden para los filtros IIR tal que el error medio cuadrático entre la señal filtrada IIR y la señal filtrada FIR sea menor que un 1% de la varianza de la señal filtrada FIR. Es decir que si $x$ es la señal filtrada IIR e $y$ es la señal filtrada FIR donde $N = \min(\text{len}(x), \text{len}(y))$ se cumpla que

$$
\sum_{i=1}^N (x_i - y_i)^2 < 0.01 \sum_{i=1}^N (y_i - \bar y_i)^2
$$

Finalmente, discuta sobre sus hallazgos muestre gráficamente el resultado del mejor filtro IIR


**Describa los filtros IIR y compárelo con los filtros FIR. Describa los argumentos y el valor de retorno de la función "scipy.signal.iirfilter"**

Los filtros IIR (respuesta infinita al impulso) se podrían decir que son una versión más avanzada de los FIR, y como su nombre lo indica la diferencia esta en su duración en el dominio de tiempo. Mientraz que un FIR es finito y eventualmente tendera a 0, el IIR usa recursión para poder seguir operando infinitamente, usando sus salidas como entradas para las siguientes operaciones.


    Parametros:
                N: Orden del filtro.
                Wn: Frecuencia(s) crítica(s).
                rp: Onda máxima para pasabanda en filtros elípticos y de Chebyshev.
                rs: Atenuación mínima para rechazabanda en filtros elípticos y de Chebyshev.
                btype: Tipo de filtro, por defecto pasabanda.
                analog: Retorna un filtro analogico si es verdadero, en otro caso digital.
                ftype: Que tipo de filtro IIR se diseñara.
                output: Que tipo de output se desea (‘ba’, ‘zpk’, ‘sos’).
                fs: Frecuencia de muestreo original.
    Retorna:
                b,a: Numerador y denominador del filtro.
                z,p,k: ceros, poles y ganancia de la función filtro.
                sos: Representación de intersecciones de segundo orden del filtro.
            




In [18]:
# btype : {'bandpass', 'lowpass', 'highpass', 'bandstop'}
bA, aA = scipy.signal.iirfilter(N=6, Wn=[59, 61], fs=Fs, ftype='butterworth', btype='bandstop') # filtro A
bB, aB = scipy.signal.iirfilter(N=6, Wn=0.9, fs=Fs, ftype='butterworth', btype='highpass') # filtro B
# filtro C

fil_iir = scipy.signal.lfilter(bA, aA, data)
fil_iir = scipy.signal.lfilter(bB, aB, data)
#fil_iir = scipy.signal.lfilter(bC, aC, data)

fig, ax = plt.subplots(figsize=(7, 3), tight_layout=True)
ax.plot(time[:3*Fs], fil_iir[:3*Fs])

<IPython.core.display.Javascript object>

In [19]:
def mse(A, B):
    return np.sum(np.square(A - B))

fir_mse = mse(data, fil_fir)

n = 1
iir_mse = 1e9
while (iir_mse >= 0.01 * fir_mse):
    bA, aA = scipy.signal.iirfilter(N=n, Wn=[59, 61], fs=Fs, ftype='butterworth', btype='bandstop') # filtro A
    bB, aB = scipy.signal.iirfilter(N=n, Wn=0.9, fs=Fs, ftype='butterworth', btype='highpass') # filtro B
    # filtro C

    fil_iir = scipy.signal.lfilter(bA, aA, data)
    fil_iir = scipy.signal.lfilter(bB, aB, data)
    #fil_iir = scipy.signal.lfilter(bC, aC, data)
    iir_mse = mse(fil_iir, fil_fir)
    n += 1
    
display(n)

NameError: name 'fil_fir' is not defined